# Tool Use

In the previous chapter, we showed you code executors which give
agents the super power of programming.
Agent writing arbitrary code is useful, however,
controling what code agent writes can be challenging. 
This is where tools come in.

Tools are pre-defined functions that agents can use. Instead of writing arbitrary
code, agents can call tools to perform actions, such as searching the web,
perform calculation, reading files, and calling remote APIs.
Because you can control what tools are available to agent, you can control what
actions agent can perform.

````mdx-code-block
:::note
Tool use is currently only available for LLMs
that support OpenAI-comaptible tool call API.
:::
````

## Creating Tool

Tools can be created as regular Python functions.
For example, let's create a calculator tool which
can only perform a single operation at a time.

In [5]:
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

The above function takes three arguments:
`a` and `b` are the integer numbers to be operated on;
`operator` is the operation to be performed.
We used type hints to define the types of the arguments and the return value.

````mdx-code-block
:::note
Always use type hints to define the types of the arguments and the return value
as they provide helpful hints to the agent about the tool's usage.
:::
````

## Registering Tool

Once you have created a tool, you can register it with the agents that
are involved in conversation.

In [29]:
import os
from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="calculator")(calculator)

<function __main__.calculator(a: int, b: int, operator: Annotated[Literal['+', '-', '*', '/'], 'operator']) -> int>

In the above code, we registered the `calculator` function as a tool with
the assistant and user proxy agents. We also provide a name and a description
for the tool for the assistant agent to understand its usage.

````mdx-code-block
:::note
Always provide a clear and concise description for the tool as it helps the
agent's underlying LLM to understand the tool's usage.
:::
````

Similar to code executor, a tool must be registered with at least two agents
for it to be useful in conversation. 
The agent registered with the tool's signature
through 
[`register_for_llm`](/docs/reference/agentchat/conversable_agent#register_for_llm)
can call the tool;
the agent registered with the tool's function object through 
[`register_for_execution`](/docs/reference/agentchat/conversable_agent#register_for_execution)
can execute the tool's function.

Alternatively, you can use 
[`autogen.register_function`](/docs/reference/agentchat/conversable_agent#register_function-1)
function to register a tool with both agents at once.

In [12]:
from autogen import register_function

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

## Using Tool

Once the tool is registered, we can use it in conversation.
In the code below, we ask the assistant to perform some arithmetic
calculation using the `calculator` tool.

In [31]:
chat_result = user_proxy.initiate_chat(assistant, message="What is (44232 + 13312 / (232 - 32)) * 5?")

User (to Assistant):

What is (44232 + 13312 / (232 - 32)) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool Call (call_bACquf0OreI0VHh7rWiP6ZE7): calculator *****
Arguments: 
{
  "a": 13312,
  "b": 232 - 32,
  "operator": "/"
}
***************************************************************************

--------------------------------------------------------------------------------
User (to Assistant):

User (to Assistant):

***** Response from calling tool "call_bACquf0OreI0VHh7rWiP6ZE7" *****
Error: Expecting ',' delimiter: line 1 column 26 (char 25)
 You argument should follow json format.
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool Call (call_2c0H5gzX9SWsJ05x7nEOVbav): calculator *****
Argumen

Let's verify the answer:

In [33]:
(44232 + int(13312 / (232 - 32))) * 5

221490

The answer is correct.
You can see that the assistant is able to understand the tool's usage
and perform calculation correctly.

## Tool Schema

If you are familiar with [OpenAI's tool use API](https://platform.openai.com/docs/guides/function-calling), 
you might be wondering
why we didn't create a tool schema.
In fact, the tool schema is automatically generated from the function signature
and the type hints.
You can see the tool schema by inspecting the `llm_config` attribute of the
agent.

In [37]:
assistant.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'A simple calculator',
   'name': 'calculator',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer', 'description': 'a'},
     'b': {'type': 'integer', 'description': 'b'},
     'operator': {'enum': ['+', '-', '*', '/'],
      'type': 'string',
      'description': 'operator'}},
    'required': ['a', 'b', 'operator']}}}]

You can see the tool schema has been automatically generated from the function
signature and the type hints, as well as the description.
This is why it is important to use type hints and provide a clear description
for the tool as the LLM uses them to understand the tool's usage.

## Summary

In this chapter, we showed you how to create, register and use tools.
Tools allows agents to perform actions without writing arbitrary code.
In the next chapter, we will introduce conversation patterns, and show
how to use the result of a conversation.